# Face Recognition using KNN
In this notebook, we aim to apply KNN based on cosine similarity distances from InceptionV3 and distance metrics from DeepFace implementations as the final culmination of the semester project.

In [3]:
# imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import cv2
from ast import literal_eval

import keras
import tensorflow as tf

In [4]:
from deepface import DeepFace

In [5]:
# file paths
path = os.getcwd()
path = path[:-4]
data_dir = path + "/data/"

### I. Load InceptionV3 Feature Embeddings from "/data/dataset_features.csv"

In [6]:
dataset = pd.read_csv(data_dir + "dataset_features.csv")

In [7]:
dataset

,Unnamed: 0,image,person,split,v3_features
0,0,[[[ 0 1 0]\n [ 0 1 0]\n [ 0 1 0]\n .....,Ann_Veneman,train,[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0...
1,1,[[[0 1 1]\n [0 1 1]\n [0 1 1]\n ...\n [1 2...,Ann_Veneman,train,[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0...
2,2,[[[0 0 0]\n [0 0 0]\n [1 0 0]\n ...\n [0 1...,Ann_Veneman,train,[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0...
3,3,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...,Ann_Veneman,train,[0.00000000e+00 0.00000000e+00 0.00000000e+00 ...
4,4,[[[129 163 176]\n [129 163 176]\n [129 163 1...,Ann_Veneman,test,[0.00000000e+00 0.00000000e+00 0.00000000e+00 ...
...,...,...,...,...,...
5090,5090,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...,Zhu_Rongji,train,[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0...
5091,5091,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...,Zhu_Rongji,train,[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0...
5092,5092,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...,Zhu_Rongji,test,[0.00000000e+00 0.00000000e+00 0.00000000e+00 ...
5093,5093,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...,Zhu_Rongji,train,[0.00000000e+00 0.00000000e+00 0.00000000e+00 ...


Let's fix the image column by rereplacing the values to image number and extract using getImg()...

In [8]:
def getImg(name, num):
    imgPath = data_dir + '/' + name + '/' + name + '_' +  "{:04d}".format(num) + '.jpg'
    img = cv2.imread(imgPath)
    img = cv2.resize(img, (299, 299))
    return img

In [11]:
def fix_feature_column(X, col):
    fixed_col = []
    for c in X[col]:
        f = [float(x) for x in c[1:-1].replace('\n', '').split()]
        fixed_col.append(f)
    return fixed_col

In [12]:
dataset["v3_features"] = fix_feature_column(dataset, "v3_features")

### II. Compute Distances (cosine sim, deepface models) of Test Data Against Train Data

### III. Get Predictions and Results